In [ ]:
#Importing Libraries
import pandas as pd

In [ ]:
#Year Data is too big so read the data in chunks in order to get a list of chunks of the data
data_iterator = pd.read_csv("2020Data_Ready.csv", chunksize=100000)

chunk_list = []  

# Each chunk is in dataframe format
for data_chunk in data_iterator:  
    chunk_list.append(data_chunk)

In [ ]:
#Choosing Specific Columns to focus on 
cl_stat1 = [] 
for chunk in chunk_list: 
    d = chunk[[ 'ean_13' , 'name' , 'Provider_Name', 'reviews_1_star',
       'reviews_2_star', 'reviews_3_star', 'reviews_4_star', 'reviews_5_star', 'review_count', 'Year', 'Month', 'Day']]
    cl_stat1.append(d)
    
    
stat1_df = pd.concat(cl_stat1)

In [ ]:
## Placing Amazon in its own dataframe as well as Good reads 
Amazon = stat1_df.loc[stat1_df['Provider_Name'] == 'Amazon']
Goodreads = stat1_df.loc[stat1_df['Provider_Name'] == 'Goodreads']

In [ ]:
#Placing all other providers in a list 
Providers = ['Apple',
 'Barnes Noble',
 'Google Play',
 'Kobo',
 'Walmart',
 'Christianbook',
 'Amazon UK']

In [ ]:
#Amazon Review Star 1 through 5 is scraped in percent form therefore a calculation is needed in order to get the correct number 
Amazon['Star1'] = (Amazon['reviews_1_star'] * Amazon['review_count']) / 100
Amazon['Star2'] = (Amazon['reviews_2_star'] * Amazon['review_count']) / 100
Amazon['Star3'] = (Amazon['reviews_3_star'] * Amazon['review_count']) / 100
Amazon['Star4'] = (Amazon['reviews_4_star'] * Amazon['review_count']) / 100
Amazon['Star5'] = (Amazon['reviews_5_star'] * Amazon['review_count']) / 100

Amazon = Amazon[['ean_13', 'name', 'Provider_Name',  'Star1', 'Star2', 'Star3', 'Star4', 'Star5',
                 'review_count','Year', 'Month', 'Day', ]]

In [ ]:
#Renaming columns in Guilford so all data can have the same column names and same order fo columns
Goodreads.rename(columns={'reviews_1_star': 'Star1',  'reviews_2_star': 'Star2', 'reviews_3_star': 'Star3' , 'reviews_4_star': 'Star4', 'reviews_5_star': 'Star5' }, inplace=True)
Goodreads = Goodreads[['ean_13', 'name', 'Provider_Name',  'Star1', 'Star2', 'Star3', 'Star4', 'Star5',
                 'review_count','Year', 'Month', 'Day']]

In [ ]:
data = [] 
for prov in Providers: 
    
    dp = stat1_df.loc[stat1_df['Provider_Name']  == prov]
    
    dp.rename(columns={'reviews_1_star': 'Star1',  'reviews_2_star': 'Star2', 'reviews_3_star': 'Star3' , 'reviews_4_star': 'Star4', 'reviews_5_star': 'Star5' }, inplace=True)
    dp = dp[['ean_13', 'name', 'Provider_Name',  'Star1', 'Star2', 'Star3', 'Star4', 'Star5',
                 'review_count','Year', 'Month', 'Day']]
    
    data.append(dp)
    print(str(prov) + ' is done')

In [ ]:
#Concating all dataframes to get one final dataframe
rest = pd.concat(data)
final = pd.concat([Amazon, Goodreads, rest])

In [ ]:
#List of providers 
providers = list(final['Provider_Name'].unique())

In [ ]:
#Stats 3 Number of New Reviews
Stats3_Data = [] 
for prov in providers: 
    
    dp = final.loc[final['Provider_Name']  == prov]
    
    months = list(dp['Month'].unique())
    
    review_count = []  
    review1star = [] 
    review2star = [] 
    review3star = [] 
    review4star = [] 
    review5star = [] 
    for month in months: 
        d = dp.loc[dp['Month'] == month]
        review_count.append(d['review_count'].sum())
        review1star.append(d['Star1'].sum())
        review2star.append(d['Star2'].sum())
        review3star.append(d['Star3'].sum())
        review4star.append(d['Star4'].sum())
        review5star.append(d['Star5'].sum())
        
    data = pd.DataFrame({'Month': months, 
                     'ReviewCount': review_count, 
                     'review_1Star': review1star,
                     'review_2Star': review2star, 
                     'review_3Star': review3star, 
                     'review_4Star': review4star, 
                     'review_5Star': review5star})
    
    
    d1 = data.set_index('Month').diff()
    d1 = d1.reset_index()
    
    d1['Provider'] = prov
    
    Stats3_Data.append(d1)
    
    print(str(prov)  + ' is done')

In [ ]:
#Concat Data and adding Domain 
Stat3_final = pd.concat(Stats3_Data)
Stat3_final['Domain'] ='All Publishers'
Stat3_final.to_csv('2020_Reviews.csv' , index = False)

In [ ]:
#Save to Excel 
with pd.ExcelWriter('AllPublishers_ReviewCount_DS79_2020.xlsx') as writer:  
    Stat3_final.to_excel(writer, sheet_name='Number of New Reviews')